In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pickle

from konlpy.tag import Okt

In [2]:
text_morph_df = pd.read_csv('morph_text.csv', encoding='utf-8')
text_nouns_df = pd.read_csv('noun_text.csv', encoding='utf-8')
label_df = pd.read_csv('classifi_corpus_label.csv', encoding='utf-8') 

In [3]:
data_df = pd.DataFrame()
data_df['ntext'] = text_nouns_df['text']
data_df['mtext'] = text_morph_df['text']
data_df['label'] = label_df['label']


In [4]:
with open('index_to_word_byOkt.pickle', 'rb') as f :
    index_to_word = pickle.load(f)
with open('index_to_noums_byOkt.pickle', 'rb') as f :
    index_to_noums = pickle.load(f)
with open('word_to_index_byOkt.pickle', 'rb') as f :
    word_to_index = pickle.load(f)
with open('noums_to_index_byOkt.pickle', 'rb') as f :
    noums_to_index = pickle.load(f)

In [5]:
(len(index_to_word), len(word_to_index)), (len(index_to_noums), len(noums_to_index))

((9997, 9997), (9999, 9999))

In [6]:
num_data_df = data_df.drop('mtext', axis= 1)
mor_data_df = data_df.drop('ntext', axis= 1)

In [7]:
num_data_df = num_data_df.dropna()
mor_data_df = mor_data_df.dropna()

In [121]:
num_data_df.ntext = num_data_df.ntext.apply(lambda x: x.split())
mor_data_df.mtext = mor_data_df.mtext.apply(lambda x: x.split())

In [122]:
#단어 -> 인덱스 변환
def n_to_int(x):
    pad_index = []
    for i in x:
        pad_index.append(noums_to_index[i])
    return pad_index

#morph -> 인덱스 변환
def w_to_int(x):
    pad_index = []
    for i in x:
        pad_index.append(word_to_index[i])
    return pad_index

In [123]:
nom_X = num_data_df.ntext.apply(lambda x: n_to_int(x))
mor_X = mor_data_df.mtext.apply(lambda x: w_to_int(x))

In [126]:
from keras.preprocessing.sequence import pad_sequences

X_nom =pad_sequences(nom_X, maxlen=15, padding='post', truncating='pre')
X_mor =pad_sequences(mor_X, maxlen=15, padding='post', truncating='pre')


print(X_nom[:5])
print(X_mor[:5])

[[2860 2329 8723 9849 4787  391    0    0    0    0    0    0    0    0
     0]
 [6440 8723 9849 4467 7516    0    0    0    0    0    0    0    0    0
     0]
 [6440 8723 9849 4467 7516    0    0    0    0    0    0    0    0    0
     0]
 [7241  359    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [3267 8723 3884 1464 3742 5649 4380    0    0    0    0    0    0    0
     0]]
[[1733 4535 3053 3843 6070  637  889  755    0    0    0    0    0    0
     0]
 [1910 3053 3843 9477 5769 4793    0    0    0    0    0    0    0    0
     0]
 [1910 3053 3843 9477 5769 4793    0    0    0    0    0    0    0    0
     0]
 [7808   61 3688    0    0    0    0    0    0    0    0    0    0    0
     0]
 [ 169 3053 4934 3923 2480 1667 2248 4346  169 3390 8004 9250 5123 7137
   158]]


In [127]:
X_nom.shape , X_mor.shape

((250614, 15), (256913, 15))

In [24]:
Y_num = num_data_df['label'].values
Y_mor = mor_data_df['label'].values

In [160]:
from sklearn.model_selection import train_test_split

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_nom, Y_num, test_size = 0.2, random_state = 77)

In [161]:
x_n_train.shape, y_n_train.shape

((200491, 15), (200491,))

In [162]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.layers import LSTM
model1 = Sequential()
model1.add(Embedding(9999, 32,input_length=15))
model1.add(LSTM(64, return_sequences= True))
model1.add(LSTM(32))
model1.add(Dense(1, activation="sigmoid"))

In [163]:
model1.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model1.fit(x_n_train, y_n_train, batch_size =128, epochs =10 , validation_split =0.2)

Train on 160392 samples, validate on 40099 samples
Epoch 1/10
160392/160392 [==============================] - 31s 193us/step - loss: 0.2424 - acc: 0.9162 - val_loss: 0.2194 - val_acc: 0.9228
Epoch 2/10
160392/160392 [==============================] - 30s 187us/step - loss: 0.2025 - acc: 0.9266 - val_loss: 0.2174 - val_acc: 0.9239
Epoch 3/10
160392/160392 [==============================] - 29s 182us/step - loss: 0.1893 - acc: 0.9307 - val_loss: 0.2166 - val_acc: 0.9246
Epoch 4/10
160392/160392 [==============================] - 30s 184us/step - loss: 0.1786 - acc: 0.9351 - val_loss: 0.2142 - val_acc: 0.9239
Epoch 5/10
160392/160392 [==============================] - 30s 185us/step - loss: 0.1680 - acc: 0.9392 - val_loss: 0.2197 - val_acc: 0.9253
Epoch 6/10
160392/160392 [==============================] - 30s 184us/step - loss: 0.1572 - acc: 0.9430 - val_loss: 0.2277 - val_acc: 0.9255
Epoch 7/10
160392/160392 [==============================] - 29s 183us/step - loss: 0.1477 - acc: 0.9464

In [164]:
from sklearn.model_selection import train_test_split

x_m_train, x_m_test, y_m_train, y_m_test = train_test_split(X_mor, Y_mor, test_size = 0.2, random_state = 77)

In [165]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.layers import LSTM
model2 = Sequential()
model2.add(Embedding(9997, 32,input_length=15))
model2.add(LSTM(64, return_sequences= True))
model2.add(LSTM(32))
model2.add(Dense(1, activation="sigmoid"))

In [168]:
model2.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
history = model1.fit(x_m_train, y_m_train, batch_size =128, epochs =10 , validation_split =0.2)

Train on 164424 samples, validate on 41106 samples
Epoch 1/10
164424/164424 [==============================] - 31s 190us/step - loss: 0.2233 - acc: 0.9185 - val_loss: 0.1929 - val_acc: 0.9250
Epoch 2/10
164424/164424 [==============================] - 30s 184us/step - loss: 0.1764 - acc: 0.9306 - val_loss: 0.1867 - val_acc: 0.9278
Epoch 3/10
164424/164424 [==============================] - 30s 184us/step - loss: 0.1557 - acc: 0.9384 - val_loss: 0.1838 - val_acc: 0.9297
Epoch 4/10
164424/164424 [==============================] - 30s 185us/step - loss: 0.1364 - acc: 0.9469 - val_loss: 0.1836 - val_acc: 0.9319
Epoch 5/10
164424/164424 [==============================] - 30s 185us/step - loss: 0.1177 - acc: 0.9553 - val_loss: 0.1846 - val_acc: 0.9360
Epoch 6/10
164424/164424 [==============================] - 30s 185us/step - loss: 0.1019 - acc: 0.9613 - val_loss: 0.1932 - val_acc: 0.9330
Epoch 7/10
164424/164424 [==============================] - 30s 185us/step - loss: 0.0885 - acc: 0.9667

In [169]:
model1.evaluate(x_n_test, y_n_test)

50123/50123 [==============================] - 5s 92us/step


[0.759630084547339, 0.863316242042192]

In [170]:
model2.evaluate(x_m_test, y_m_test)

51383/51383 [==============================] - 5s 98us/step


[0.6969192079747171, 0.17692622072868414]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_tfidf = TfidfVectorizer()
noum_tfidf = TfidfVectorizer()

In [9]:
mor_word=list(word_to_index.keys())
nou_word=list(noums_to_index.keys())

In [10]:
word_tfidf.fit(mor_word)
noum_tfidf.fit(nou_word)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
len(word_tfidf.vocabulary_) ,len(noum_tfidf.vocabulary_)

(9285, 9199)

In [12]:
num_data_df = data_df.drop('mtext', axis= 1)
mor_data_df = data_df.drop('ntext', axis= 1)

num_data_df = num_data_df.dropna()
mor_data_df = mor_data_df.dropna()

In [13]:
tf_m_X = mor_data_df.mtext.apply(lambda x : word_tfidf.transform([x]).toarray())


In [14]:
tf_n_X = num_data_df.ntext.apply(lambda x : noum_tfidf.transform([x]).toarray())

In [40]:
tf_m_X.shape, tf_m_X[0].shape 

((256913,), (1, 9285))

In [78]:
data= []
for sentence in mor_data_df.mtext:
        # 특수기호 제거
        
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        
        data.append(sentence)

data

['저 직업 고민 상담 쫌 해주세요 ㅜㅜㅜ 제발',
 '으로 고민 상담 해주는 사이트 좀',
 '으로 고민 상담 해주는 사이트 좀',
 '주변 에 별로',
 '한테 고민 털어 내 라거 나 학교 선생님 한테 물어보면 추천 답글 은 없으면 좋겠어요',
 '고민 상담 전화',
 '고민 상담 전화',
 '친구 관계 거든요 애 들 이랑 싸워서 힘든 내용 인데',
 '저 의 나 그 애 들 정보 그리고 소문 이런거 나요 제 전번 남아 요',
 '학교 에 나 부모님 선생님 연락 없죠 그거 어디 다가 하는건 지도 빨리 알려주세여 ㅠㅠ',
 '입 시생 고민 상담',
 '너무 걱정 이 들고 눈물 도 나오네요 할 수 있을까요 제 자신 을 너무 몰라서요',
 '고민 상담 은 삼촌',
 '고민 상담 할 만 한 곳',
 '사귀는거 공부 문제 등 이에요 제 가 어디 에다 고민 을 상담 받을 수 있을까요',
 '고민 상담 좀 해주세요',
 '씩 일 하고 교대 근무 하면서 한 달 에 만원 정도 버 는 살 입니다',
 '그런데 문제 는 집 에 빚 도 많고 돈 이 과자 도 없어서',
 '인터넷 도 끊 키 고 수도 가스 전기 전부 다',
 '제 가 을 물론 맞지만 너무나 당연하게',
 '생각 하는듯 하여 제 가 너무 나 에게 의존 하는거 아니냐',
 '라고 오히려 저 에게 성질 을 내면 서 전화 를',
 '끊어 저 에게 정말 현실 적 인 답좀',
 '진로 진학 고민 상담',
 '꿈 을 뒷 할 정도 로 공부 를 하지 못 해서 일단 그 꿈 은',
 '꼭 알려주세요',
 '학과 고민 상담',
 '해서 강제 로 인생 이 답 이 없네요 제 가 무슨 말 을 하는지도 모르겠네요',
 '그냥 조용히 년 까요',
 '고민 상담 사이트',
 '고민 상담 사이트 중 에 친구 한테 들었는데 라는 게',
 '아시 는 분 혹시 사이트 주소 나 정보 좀 알려주실 수 있나요',
 '고민 상담',
 '학년 여고생 인데 요 제 가 결정 한 길 에 대해 서 고민 이 많아서',
 '그런데 고민 상담 할 수 있는 곳 이 있을까요 웬만

In [87]:
test_tf = TfidfVectorizer()

In [88]:
test_tf.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [92]:
test_tf.transform(data).toarray()

MemoryError: 

In [25]:
from sklearn.model_selection import train_test_split

x_m_train, x_m_test, y_m_train, y_m_test = train_test_split(tf_m_X, Y_mor, test_size = 0.2, random_state = 77)
x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(tf_n_X, Y_num, test_size = 0.2, random_state = 77)